In [50]:
# Kaggle: https://www.kaggle.com/competitions/llm-classification-finetuning/
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [51]:
train=pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')
test=pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')

In [52]:
train.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [53]:
test.head()

,id,prompt,response_a,response_b
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran..."
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ..."
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p..."


In [54]:
train.drop(columns=['model_a','model_b','id'], inplace=True)
train.head()

,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [55]:
train.tail()

,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
57472,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0
57473,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0
57474,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0
57475,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0
57476,"[""three kids eat three apples in three days, h...","[""27 apples""]","[""If three kids eat three apples in three days...",1,0,0


In [56]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57477 entries, 0 to 57476
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   prompt          57477 non-null  object
 1   response_a      57477 non-null  object
 2   response_b      57477 non-null  object
 3   winner_model_a  57477 non-null  int64 
 4   winner_model_b  57477 non-null  int64 
 5   winner_tie      57477 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 2.6+ MB


In [57]:
import re

def clean_text(text):
    if pd.isnull(text):
        return ""
    text = str(text)
    
    # Basic cleaning
    text = text.encode('utf-8').decode('unicode_escape', errors='ignore')
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = text.replace('\"', '"').replace("\'", "'")
    text = text.replace('"', '').replace('[', '').replace(']', '').replace(',', '')

    # Lowercase
    text = text.lower()
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', ' ', text)

    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    
    # Optional: remove numbers
    # text = re.sub(r'\d+', '', text)

    # Trim
    text = text.strip()
    
    return text


# Apply cleaning to all text columns
for col in ['prompt', 'response_a', 'response_b']:
    train[col] = train[col].apply(clean_text)
    test[col] = test[col].apply(clean_text)    

/tmp/ipykernel_31/2595280604.py:9: DeprecationWarning: invalid escape sequence '\/'
  text = text.encode('utf-8').decode('unicode_escape', errors='ignore')


In [58]:
train.tail()

,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
57472,a simple mnemonic for π how i wish i could enu...,sure let s break it down 1 how has 3 letters 2...,here is how that mnemonic represents the digit...,1,0,0
57473,in python implement a naive bayes with gaussia...,here is an implementation of a naive bayes cla...,sure here s an implementation of a naive bayes...,1,0,0
57474,is it unethical to work on building weapons ar...,working on weapons technology raises some ethi...,it depends on the context weapons can be used ...,1,0,0
57475,if a bait contains 00025 bromadiolon then how ...,bromadiolone is a rodenticide which is most of...,as an ai language model i do not promote or co...,0,1,0
57476,three kids eat three apples in three days how ...,27 apples,if three kids eat three apples in three days e...,1,0,0


In [59]:
train.head()

,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,is it morally right to try to have a certain p...,the question of whether it is morally right to...,as an ai i don t have personal beliefs or opin...,1,0,0
1,what is the difference between marriage licens...,a marriage license is a legal document that al...,a marriage license and a marriage certificate ...,0,1,0
2,explain function calling how would you call a ...,function calling is the process of invoking or...,function calling is the process of invoking a ...,0,0,1
3,how can i create a test set for a very rare ca...,creating a test set for a very rare category c...,when building a classifier for a very rare cat...,1,0,0
4,what is the best way to travel from tel aviv t...,the best way to travel from tel aviv to jerusa...,the best way to travel from tel aviv to jerusa...,0,1,0


In [60]:
test.head()

,id,prompt,response_a,response_b
0,136060,i have three oranges today i ate an orange yes...,you have two oranges today,you still have three oranges eating an orange ...
1,211333,you are a mediator in a heated political debat...,thank you for sharing the details of the situa...,mr reddy and ms blue both have valid points in...
2,1233961,how to initialize the classification head when...,when you want to initialize the classification...,to initialize the classification head when per...


In [61]:
max_words = train['response_a'].apply(lambda x: len(str(x).split())).max()
print("Maximum number of words in any row:", max_words)

Maximum number of words in any row: 9501


#  TF-IDF + Truncated SVD (Latent Semantic Analysis) + N-grams

✅ Captures hidden relationships (like "king" ≈ "queen" somehow)

Why we combined the 3 columns for TF-IDF (Bag-of-Words model - Word importance only	- NOT Meaningful sentence embeddings):

    TF-IDF works on bag of words — it doesn't know sentence structure, order, or meaning.

    TF-IDF just cares about: "which words exist, and how frequent?"

    So if you split into separate columns, TF-IDF will treat them independently, and waste information.

    Therefore, to capture full word context, we combine 'prompt', 'response_a', and 'response_b' into a single text for TF-IDF.
    → Then TF-IDF sees all words together and computes the correct word weights across the combined context.

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack  

# Combine all text columns into one series for fitting
combined_text_train = train['prompt'] + ' ' + train['response_a'] + ' ' + train['response_b']
combined_text_test = test['prompt'] + ' ' + test['response_a'] + ' ' + test['response_b']

'''
# TF-IDF only
vectorizer = TfidfVectorizer(max_features=10000)   

# Fit and transform
X_train_combined = vectorizer.fit_transform(combined_text_train)
X_test_combined = vectorizer.transform(combined_text_test)

# These are your final feature matrices
final_train_features = X_train_combined
final_test_features = X_test_combined
'''

from sklearn.decomposition import TruncatedSVD

# TF-IDF + n-gram
vectorizer = TfidfVectorizer(max_features=30000, ngram_range=(1,2))
X_train_combined = vectorizer.fit_transform(combined_text_train)
X_test_combined = vectorizer.transform(combined_text_test)

# Truncated SVD
svd = TruncatedSVD(n_components=300, random_state=42)   
X_train_svd = svd.fit_transform(X_train_combined)
X_test_svd = svd.transform(X_test_combined)

final_train_features = X_train_svd
final_test_features = X_test_svd


In [66]:
from lightgbm import LGBMClassifier, early_stopping
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_predict, StratifiedKFold

# Map the labels to numerical values (0: winner_model_a, 1: winner_model_b, 2: winner_tie)
y = train[['winner_model_a', 'winner_model_b', 'winner_tie']].idxmax(axis=1).map({'winner_model_a': 0, 'winner_model_b': 1, 'winner_tie': 2})

X = final_train_features

In [74]:
X

array([[ 0.33207334,  0.02913317,  0.01880533, ..., -0.0445559 ,
         0.07589928, -0.03753681],
       [ 0.23079724,  0.00096624, -0.03016245, ...,  0.00855667,
        -0.0114794 , -0.01806916],
       [ 0.26256172, -0.06968897, -0.15364821, ...,  0.01698376,
         0.02001671,  0.0183725 ],
       ...,
       [ 0.34269937,  0.03865303,  0.01756029, ..., -0.00663313,
        -0.01787836,  0.01837855],
       [ 0.28076092,  0.01727467, -0.03398818, ...,  0.00267414,
        -0.01881258, -0.01036943],
       [ 0.04053574,  0.00842621,  0.00371777, ..., -0.00353098,
         0.0169585 , -0.01647963]])

In [75]:
y

0        0
1        1
2        2
3        0
4        1
        ..
57472    0
57473    0
57474    0
57475    1
57476    0
Length: 57477, dtype: int64

In [67]:
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.model_selection import ParameterSampler
# 1. Define search space
param_grid = {
    'min_child_samples': [10, 25, 50],
    'n_estimators': [500, 1000],    # Number of boosting rounds
    'max_depth': [10, 15],        # Limit the depth of trees
}

n_trials = 5  # How many hyperparameter combinations you want to try
param_sampler = list(ParameterSampler(param_grid, n_iter=n_trials, random_state=42))

# 2. Track best model
best_val_loss = np.inf
best_model = None
best_params = None

fold = 0
train_losses = []
val_losses = []
models = []

# Set up cross-validation
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for params in param_sampler:
    print(f"\n🎯 Trying parameters: {params}")
    
    fold = 0
    fold_train_losses = []
    fold_val_losses = []
    
    for train_idx, val_idx in cv.split(X, y):
        fold += 1
        print(f"\n🔁 Fold {fold}")

        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # Initialize model with sampled hyperparameters
        model = LGBMClassifier(
            objective='multiclass',
            num_class=3,
            random_state=42,
            eval_metric='multi_logloss',
            bagging_freq=5,
            lambda_l2  = 0.5,
            lambda_l1  = 0.5,
            **params  # <<< inject sampled params here
        )
        
        model.fit(
            X_train, y_train,
            eval_set=[(X_train, y_train), (X_val, y_val)],
            eval_names=['train', 'valid'],
            callbacks=[
                early_stopping(stopping_rounds=20),
                log_evaluation(period=100)
            ]
        )

        # Predict probabilities
        train_probs = model.predict_proba(X_train)
        val_probs = model.predict_proba(X_val)

        # Compute log loss
        train_logloss = log_loss(y_train, train_probs)
        val_logloss = log_loss(y_val, val_probs)

        print(f"📊 Fold {fold} Log Loss - Train: {train_logloss:.4f}, Validation: {val_logloss:.4f}")
        fold_train_losses.append(train_logloss)
        fold_val_losses.append(val_logloss)

    avg_val_loss = np.mean(fold_val_losses)
    print(f"\n🧠 Average Validation Log Loss for this parameter set: {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model = model
        best_params = params

    models.append(model)
    train_losses.append(np.mean(fold_train_losses))
    val_losses.append(avg_val_loss)

# 🔥 Best results
print("\n✅ Hyperparameter Search Finished")
print(f"Best Validation Log Loss: {best_val_loss:.4f}")
print(f"Best Parameters: {best_params}")


🎯 Trying parameters: {'n_estimators': 500, 'min_child_samples': 50, 'max_depth': 15}

🔁 Fold 1
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.081095 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGB

In [68]:
print(train_losses)
print(val_losses)

[1.0168071309667794, 1.0103297012343697, 1.0174670196476874, 1.0103297012343697, 1.0067726632650664]
[1.084561788133965, 1.0840695212276608, 1.0841724213851165, 1.0840695212276608, 1.0838798875059048]


In [69]:
test_probs = best_model.predict_proba(final_test_features)  # test_features is the feature matrix for your test set

# Predict the class labels on the test data
test_labels = best_model.predict(final_test_features)

# Display the predicted probabilities (for each class)
print("Test Probabilities:")
print(test_probs)

# Display the predicted class labels (0, 1, or 2)
print("Test Class Labels:")
print(test_labels)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
Test Probabilities:
[[0.26555315 0.27666264 0.45778421]
 [0.42781882 0.27943092 0.29275027]
 [0.35428346 0.3664045  0.27931204]]
Test Class Labels:
[2 0 1]


In [70]:
prob_df = pd.DataFrame(test_probs, columns=['prompt', 'response_a', 'response_b'])
		
output = pd.concat([test['id'].reset_index(drop=True), prob_df], axis=1)
#output.to_csv('submission.csv', index=False)
output.head()

,id,prompt,response_a,response_b
0,136060,0.265553,0.276663,0.457784
1,211333,0.427819,0.279431,0.292750
2,1233961,0.354283,0.366404,0.279312


### bag of words models are not giving very good score. Thus, meaningful emdeddings required:

In [80]:
import gc
del prob_df, test_probs, test_labels, final_train_features, X_train_svd, final_test_features, X_test_svd, X_train_combined, X_test_combined 
gc.collect()

636

# trying embeddings as they also capture context 
## Con: takes too much memory that's why encoding embeddings in batches otherwise causing RAM full and crash

In [77]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-albert-small-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/827 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Why we encode separately for SentenceTransformer:

    Sentence Transformers (like 'paraphrase-albert-small-v2') understand sentences and semantic meaning.

    They generate one meaningful vector per input (whether it's a prompt, a response_a, or a response_b).

    If you blindly combine 'prompt' + 'response_a' + 'response_b' into one text,
    ➔ you will lose individual semantic meanings.
    (e.g., model won't know which part is prompt and which is response)

Instead:

    You separately encode 'prompt', 'response_a', and 'response_b' into separate embeddings. Then concatenate the embeddings - This way you preserve the meaning of each part separately and let the model learn relationships between them!

In [ ]:
prompt_embeds = model.encode(train['prompt'].tolist(), batch_size=2048, show_progress_bar=True)

In [81]:
prompt_embeds

array([[ 0.23200333,  0.58380276, -0.01959007, ..., -0.28125316,
         0.5319085 , -0.49500197],
       [ 0.13280816,  0.14608544, -0.44802836, ...,  0.90225863,
         0.21633628,  0.3787431 ],
       [-0.20034552, -0.6754557 ,  1.1417165 , ...,  0.31891087,
        -0.00328754, -1.6726645 ],
       ...,
       [-0.28393358,  0.3422195 ,  0.5409369 , ...,  0.01849821,
         0.33610758, -0.2706519 ],
       [-0.02055523,  0.12156801, -0.66176605, ..., -0.46185187,
        -0.28438604, -0.15869723],
       [-0.3545313 ,  1.141768  , -1.1357027 , ...,  0.5116905 ,
         0.993347  , -0.06641866]], dtype=float32)

In [84]:
print(prompt_embeds.shape)

(57477, 768)


In [86]:
# Encode each column separately
response_a_embeds = model.encode(train['response_a'].tolist(), batch_size=2048, show_progress_bar=True)
response_b_embeds = model.encode(train['response_b'].tolist(), batch_size=2048, show_progress_bar=True)

# Stack horizontally
X = np.hstack([prompt_embeds, response_a_embeds, response_b_embeds])


Batches:   0%|          | 0/29 [00:00<?, ?it/s]

Batches:   0%|          | 0/29 [00:00<?, ?it/s]

In [87]:
del prompt_embeds, response_a_embeds, response_b_embeds, train
gc.collect()

17

In [91]:
fold = 0
train_losses = []
val_losses = []
models = []

cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

for train_idx, val_idx in cv.split(X, y):
    fold += 1
    print(f"\n🔁 Fold {fold}")

    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model_lgbm = LGBMClassifier(
        objective='multiclass',
        num_class=3,
        n_estimators=1000,
        random_state=42,
        max_depth=10,
        min_child_samples=50,
        lambda_l1=0.5,
        lambda_l2=0.5,
        bagging_freq=5,
        eval_metric='multi_logloss'
    )

    model_lgbm.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        eval_names=['train', 'valid'],
        callbacks=[
            early_stopping(stopping_rounds=20),
            log_evaluation(period=100)
        ]
    )

    models.append(model_lgbm)

    # Predict probabilities
    train_probs = model_lgbm.predict_proba(X_train)
    val_probs = model_lgbm.predict_proba(X_val)

    # Compute log loss
    train_logloss = log_loss(y_train, train_probs)
    val_logloss = log_loss(y_val, val_probs)

    print(f"📊 Fold {fold} Log Loss - Train: {train_logloss:.4f}, Validation: {val_logloss:.4f}")
    train_losses.append(train_logloss)
    val_losses.append(val_logloss)

# Summary
print("\n✅ Cross-validated Results")
print(f"Mean Training Log Loss: {np.mean(train_losses):.4f}")
print(f"Mean Validation Log Loss: {np.mean(val_losses):.4f}")



🔁 Fold 1
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.999549 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 587520
[LightGBM] [Info] Number of data points in the train set: 

## Despite very high computational cost (many times higher) - it is not doing much better than TF-IDF.

In [ ]:
# Similarly for test
prompt_embeds_test = model.encode(test['prompt'].tolist(), batch_size=2048, show_progress_bar=True)
response_a_embeds_test = model.encode(test['response_a'].tolist(), batch_size=2048, show_progress_bar=True)
response_b_embeds_test = model.encode(test['response_b'].tolist(), batch_size=2048, show_progress_bar=True)

test.drop(columns=['response_a','response_b','prompt'], inplace=True)

X_test = np.hstack([prompt_embeds_test, response_a_embeds_test, response_b_embeds_test])

del prompt_embeds_test, response_a_embeds_test, response_b_embeds_test
gc.collect()

In [ ]:
test_probs = best_model.predict_proba(X_test)  # test_features is the feature matrix for your test set

# Predict the class labels on the test data
test_labels = best_model.predict(X_test)

# Display the predicted probabilities (for each class)
print("Test Probabilities:")
print(test_probs)

# Display the predicted class labels (0, 1, or 2)
print("Test Class Labels:")
print(test_labels)

In [ ]:
prob_df = pd.DataFrame(test_probs, columns=['prompt', 'response_a', 'response_b'])
		
output = pd.concat([test['id'].reset_index(drop=True), prob_df], axis=1)
output.to_csv('submission.csv', index=False)
output.head()